Метою даного проекту є спроба створення, з наявних даних, моделі машинного навчання для детекції хибних кросс-посилань (аналогів деталей в автомобільній сфері) і виявлення нових крос-посилань. 

Для дослідження була обрана категорія деталей "бендикси" (комплектуюча стартера) по принципу найбільшої кількості заповнених параметрів без хаосу в самій ієрархії параметрів. Оцінка проводлася в excel для швидкості прийнятяя рішення по категорії для дослідження.

Взявши всі наявні артикули з корпоративної бази даних (hashed_article.xlsx) я попередньо захешував їх для забезпечення безпеки корпоративних даних. Також для створення моделі взяв параметри (hashed_param_bend.xlsx) і крос-посилання (hashed_cross_bend.xlsx) бендиксів. 

Всього артикулів - 38604 шт.
Значень параметрів - 18095 шт.
Крос-посиласнь - 9454 шт.

Тобто з самого початку помітна дуже низька кількість даних, та все ж основна частина з параметрами - це ті товари, які мають рух по продажам за останні 5, відповідно, більш актуальні позиції. 

Далі, зробивши pivot-таблицю і оцінивши кількість параметрів які заповнені, я видалив деякі наперед не придатні для моделі по причині недостатку даних. 
В наступному кроці я видалив нульові і некоректні дані з параметрів і знову видалив малоінформативні параметри. Всього вийшло 1794 артикулів, які були з заповненими параметрами. 

З допомогою матриці кореляцій і pairplot, значних кореляцій між параметрами не було виявлено, тож я залишив всі параметри для тесту моделі. З допогою Q-Q Plot я оцінив нормальність розподілу даних в кожному параметрі. Розподіл не у всіх параметрах був задовільний, але спробувати варто. Можливо, в подальшому застосую якийсь певний вид нормалізації даних, якщо модель буде погано описувати результат (правдивий крос чи ні). 

Підтягнувши вичищені дані параметрів в таблицю правдивих крос-посилань і видаливши записи без параметрів, в мене залишилося вже 402 кроси.

За основу для майбутньої моделі я вивів різницю в однакових параметрах для зкросованих бендиксів.

Після цього, я згенерував з наявних товарів з параметрами нові рандомні крос-посилання (таку ж кількість як і правдивих), але такі, яких не було раніше в списку правдивих кросів (як А на В так і В на А). Далі об'єднав правдиві і не правдиві кроси в один датафрейм.

Оцінивши знову дані на залежність між параметрами і нормальність розподілу, картина погіршилася, так як дуже маленький набір товарів з характеристиками, відповідно, певні записи характеристик мали вже більшу вагу.

Побудувавши boxplot в зрізі правдивий крос чи ні, видно, що відмінність в параметрах більшке 0 в кількості зубів і шліців є критичним, в інших випадках не однозначно, так як довірчі інтервали в різницях правдивих кросів захоплюють довірчі інтервали не правдивих кросів. Залежність є багатофакторою, що є дуже логічним з самого початку. 

Обравши логістичну регресію як базову, найпростішу, вона видала результат точності в 98%, що є дуже хорошим показником, як на мене. Дані для тренування складали 80% від підготовленої вибірки, а для тесту - 20% і стратифіковані по результату (правильний чи не правильний крос).

Далі я спробував інші моделі (дерево рішень, Bagging з базовим оцінювачем LogisticRegression, метод опорних векторів з лінійним ядром), але супер приросту вони не дали. Лише ансамбль моделей LogisticRegression і SGDClassifier дали точність на декілька десятих відсотка більше.

Так як даних більше в мене нема, а дуже хочеться протестувати роботоздатність виявлення кросів, я згенерував всіможливі комбінації кросів з артикулів, які в мене були з параметрами (окрім тих, які були у правильних кросах) і застосував до них мою модель. Тобто теоретично, майже всі кроси мали б бути неправдивими. 

Модель підтвердила неправдивість кросів для 26964 записів, але і 1124 записи (менше 5%) виявила як правдиві. 
Оцінивши їх з експертом по такого роду деталям, виявилось, що кроси, які видала модель як правдиві - не правильні в більшості випадків. При порівнянні бендиксів, які мали бути аналогічними, виявилось, що основні параметри які є в моделі +- схожі але є ще додаткові параметри яких немає в базі даних, але вони є важливими. 

Виходячи з наявних даних і результатів, маю такі висновки:
Хоч модель мала високий рівень точності (98 %) на тестових даних, як інструмент для пошуку нових крос-посилань поки її використовути, нажаль, не можна. Протестувавши запропоновані крос-посилання виявилась деяка кількість параметрів яких поки впринципі нема в базі даних, так як вони не замірялися раніше.

Проте, як інструмент для грубої перевірки крос-посилань на наявність грубої помилки - модель можна використувати, так як показала при перевірці експертом хороший результат (менше ніж 5% похибки в виявленні помилкових значень).

Впевнений, додавши в модель відсутні параметри - вона може згодитися навіть і для формування нових кросів. Але це вже інша історія. Потрібно більше даних, а відповідно і ресурс для реалізації задуму, який попередньо треба ще оцінити з точки зору профіту і пріоритетності задач.






